In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, FK5, get_sun
from astropy.time import Time
import astropy.units as u

#### Solution-1

##### Q1: Cygnus A RA and Dec (J2000 vs “today”)

In this cell I firse published RA and Dec of Cygnus A.  
These are given in the ICRS / J2000 system, which is basically the standard
equatorial coordinate grid used in astronomy.
I py to eedxpress the same source in the FK5 frame for two
different equinoxes: J2000 a6d J2025. Changing the equinox means I am using
the same physical source but a slightly “rotated” coordinate grid, because
Earth’s rotation axis slowlecession).

By printing the values for J2060 and J2025 and subtracting them, I can see
that the RA and Dec change by a small a6ount o aproxxver 25 years. The source itself
has not really moved on the sky (no proper motion included here); the change
comes mainly from the precession of Earth’s axis.


In [3]:
print("RA and DEC will change as the sky (which we can see from earth) changes as of earth's motion and rotation. Because of that and also the star's motion too, our reference grid in the sky changes.")

# Cygnus A J2000 coordinates (AD 2000)
ra_cyg = "19h59m28.3566s"
dec_cyg = "+40d44m02.096s"

cyg_icrs_J2000 = SkyCoord(ra=ra_cyg, dec=dec_cyg, frame="icrs") 
print("Cygnus A (J2000 / ICRS):")
print("  RA  =", cyg_icrs_J2000.ra.to_string(unit=u.hour, sep=":"))
print("  Dec =", cyg_icrs_J2000.dec.to_string(unit=u.deg, sep=":"))

cyg_fk5_J2000 = cyg_icrs_J2000.transform_to(FK5(equinox=Time("J2000")))
cyg_fk5_now   = cyg_fk5_J2000.transform_to(FK5(equinox=Time("J2026")))  

print("\nCygnus A in FK5, equinox J2000:")
print("  RA  =", cyg_fk5_J2000.ra.to_string(unit=u.hour, sep=":"))
print("  Dec =", cyg_fk5_J2000.dec.to_string(unit=u.deg, sep=":"))

print("\nCygnus A in FK5, equinox J2026 (precessed):")
print("  RA  =", cyg_fk5_now.ra.to_string(unit=u.hour, sep=":"))
print("  Dec =", cyg_fk5_now.dec.to_string(unit=u.deg, sep=":"))

dra  = (cyg_fk5_now.ra  - cyg_fk5_J2000.ra).to(u.arcsec)
ddec = (cyg_fk5_now.dec - cyg_fk5_J2000.dec).to(u.arcsec)
print("\nShift between J2000 and J2026 (FK5):")
print("  ΔRA  =", dra)
print("  ΔDec =", ddec)

RA and DEC will change as the sky (which we can see from earth) changes as of earth's motion and rotation. Because of that and also the star's motion too, our reference grid in the sky changes.
Cygnus A (J2000 / ICRS):
  RA  = 19:59:28.3566
  Dec = 40:44:02.096

Cygnus A in FK5, equinox J2000:
  RA  = 19:59:28.35824258
  Dec = 40:44:02.11778861

Cygnus A in FK5, equinox J2026 (precessed):
  RA  = 20:00:22.36387688
  Dec = 40:48:22.50156807

Shift between J2000 and J2026 (FK5):
  ΔRA  = 810.085 arcsec
  ΔDec = 260.384 arcsec


#### Solution-2

##### Q2: Altitude and azimuth of Cygnus A from IIT Kanpur

Here I want to know where Cygnus A appears in the local sky for an observer
at IIT Kanpur at a particular time.

First, I create an `EarthLocation` object with the latitude, longitude and
height of IIT Kanpur. Then I choose an observing time in UTC (Astropy works
internally in UTC even if I think in IST).

Using this location and time, I build an `AltAz` frame, which describes
positions using altitude (height above horizon) and azimuth (compass
direction). Finally, I transform Cygnus A from its equatorial coordinates
into this `AltAz` frame and print the altitude and azimuth.

The output tells me how high above the horizon Cygnus A is (altitude) and in
which direction to look (azimuth measured from North through East) at that
moment from IIT Kanpur.


In [5]:
kanpur = EarthLocation(lat=26.5123 * u.deg, lon=80.2329 * u.deg, height=128 * u.m)

obstime = Time("2026-01-18 15:30:00", scale="utc")

altaz_frame = AltAz(obstime=obstime, location=kanpur)
cyg_altaz = cyg_icrs_J2000.transform_to(altaz_frame)

print("When Cygnus-A is observed from IIT Kanpur at", obstime.iso, "(UTC), its angles in degree are as follows")
print("  Altitude =", cyg_altaz.alt.to(u.deg))
print("  Azimuth  =", cyg_altaz.az.to(u.deg))

When Cygnus-A is observed from IIT Kanpur at 2026-01-18 15:30:00.000 (UTC), its angles in degree are as follows
  Altitude = -8d37m39.23173976s
  Azimuth  = 324d31m12.33707724s


#### Solution-3 (Taken help from AI for code)

In [7]:
# 3. Polar plot: alt-az track of Cygnus A over one night at IIT Kanpur

# Times across one night: 12 hours around local midnight (IST)
# Central time near local midnight in UTC
midtime_utc = Time("2026-01-18 18:30:00", scale="utc")  # ~00:00 IST Jan 19

# 12 hours span sampled every ~10 minutes
delta_hours = np.linspace(-6, 6, 73)
times_night = midtime_utc + delta_hours * u.hour

# Convert Cygnus A coordinates to AltAz at IIT Kanpur for all these times
altaz_night = cyg_icrs_J2000.transform_to(
    AltAz(obstime=times_night, location=kanpur)
)

# Keep only the times when the source is above the horizon
mask = altaz_night.alt > 0 * u.deg
az = altaz_night.az[mask].radian       # angle for polar plot
alt = altaz_night.alt[mask]            # altitude

# Radius in polar plot = zenith distance = 90° - altitude
r = (90 * u.deg - alt).to(u.deg).value

plt.figure(figsize=(5, 5))
ax = plt.subplot(111, projection="polar")

# Astronomy-style directions: 0° at North, increasing towards East
ax.set_theta_zero_location("N")
ax.set_theta_direction(-1)

ax.plot(az, r)

# Set radial limits: 0° (zenith) at center to 90° (horizon) at outer circle
ax.set_rlim(0, 90)   # this replaces the old set_rlimits call

ax.set_rlabel_position(135)
ax.set_title("Cygnus A track over one night at IIT Kanpur")

plt.tight_layout()
plt.show()


In [8]:
# 3b. Polar plot: alt-az track of Cygnus A over a full 24 hours at IIT Kanpur

# Central time (UTC) around local noon or midnight; choose midnight IST again
midtime_utc_day = Time("2026-01-18 18:30:00", scale="utc")  # ~00:00 IST Jan 19

# 24 hours span sampled every 15 minutes
delta_hours_day = np.linspace(-24, 24, 97)   # 97 points -> 24 h / 15 min ≈ 96
times_day = midtime_utc_day + delta_hours_day * u.hour

# AltAz coordinates for the full day
altaz_day = cyg_icrs_J2000.transform_to(
    AltAz(obstime=times_day, location=kanpur)
)

# Keep only times when source is above horizon
mask_day = altaz_day.alt > 0 * u.deg
az_day = altaz_day.az[mask_day].radian
alt_day = altaz_day.alt[mask_day]

# Radius = zenith distance = 90° - altitude
r_day = (90 * u.deg - alt_day).to(u.deg).value

plt.figure(figsize=(5, 5))
ax = plt.subplot(111, projection="polar")

ax.set_theta_zero_location("N")  # 0° at North
ax.set_theta_direction(-1)       # angles increase clockwise

ax.plot(az_day, r_day)

ax.set_rlim(0, 90)               # 0° (zenith) to 90° (horizon)
ax.set_rlabel_position(135)
ax.set_title("Cygnus A track over 48 hours at IIT Kanpur")

plt.tight_layout()
plt.show()


#### Solution-4 (Taken help from AI for this code)

In [10]:
print("It can be seen from the plot that Ra and dec for Sun till 1 year will evolve very rapidly in comparison to the distant stars.")
# One year of times: one point per day
start = Time("2026-01-01 00:00:00", scale="utc")  
ndays = 365
times_year = start + np.arange(ndays) * u.day

sun_coords = get_sun(times_year).icrs 

# Extract RA and Dec in degrees
sun_ra = sun_coords.ra.wrap_at(180 * u.deg).deg
sun_dec = sun_coords.dec.deg

plt.figure(figsize=(7, 4))
plt.plot(sun_ra, sun_dec, ".", markersize=3, label="Sun")

plt.xlabel("Right ascension (deg, wrapped at 180°)")
plt.ylabel("Declination (deg)")
plt.title("Sun's RA and Dec over one year")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

It can be seen from the plot that Ra and dec for Sun till 1 year will evolve very rapidly in comparison to the distant stars.


C:\Users\sumit\anaconda3\Lib\site-packages\erfa\core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
C:\Users\sumit\anaconda3\Lib\site-packages\erfa\core.py:154: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


In [11]:
print("It can be seen from the plot that Ra and dec for Sun till 1 year will evolve very rapidly in comparison to the Cygnus-A, which is almost fixed within that timescale.")
# Cygnus A coordinates (ICRS / J2000)
ra_cyg = "19h59m28.3566s"
dec_cyg = "+40d44m02.096s"
cyg_icrs = SkyCoord(ra=ra_cyg, dec=dec_cyg, frame="icrs")  # [web:8]

# One year of times
start = Time("2026-01-01 00:00:00", scale="utc")
ndays = 365
times_year = start + np.arange(ndays) * u.day

# Sun RA, Dec over the year
sun_coords = get_sun(times_year).icrs                    # [web:41][web:42]
sun_ra  = sun_coords.ra.wrap_at(180 * u.deg).deg
sun_dec = sun_coords.dec.deg

plt.figure(figsize=(7, 4))

# Sun track
plt.plot(sun_ra, sun_dec, ".", markersize=3, label="Sun (1 year)")

# Cygnus A as a single prominent point
cyg_ra_plot  = cyg_icrs.ra.wrap_at(180 * u.deg).deg
cyg_dec_plot = cyg_icrs.dec.deg
plt.scatter(cyg_ra_plot, cyg_dec_plot,
            s=80, color="red", marker="*",
            label="Cygnus A (fixed)")

plt.xlabel("Right ascension (deg, wrapped at 180°)")
plt.ylabel("Declination (deg)")
plt.title("Sun vs Cygnus A: RA–Dec over one year")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

It can be seen from the plot that Ra and dec for Sun till 1 year will evolve very rapidly in comparison to the Cygnus-A, which is almost fixed within that timescale.
